In [33]:
import psycopg2
import random
import names
import uuid
import datetime

In [48]:
def index_tester(what_index, Query, Drop):
    try:
        con = psycopg2.connect(
                host = "127.0.0.11",
                database="ZSBD_p1",
                user = "inzdanych123",
                password = "inzdanych123")

        cur = con.cursor()

        cur.execute(Drop)

        if what_index != None:
            cur.execute(what_index)

        cur.execute(Query)
        query_output = cur.fetchall()

        for row in query_output:
            print(row)

    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL", error)

    finally:
        # closing database connection.
        if con:
            cur.close()
            con.close()

In [49]:
Q = 'EXPLAIN ANALYSE SELECT * FROM ai_med."leczenie_w_kierunku_kod_ICD_10"' + " WHERE data_zakonczenia_leczenia_i10 >= '2021-02-01' AND data_zakonczenia_leczenia_i10 <= '2021-02-28';"
D = 'DROP INDEX IF EXISTS ai_med.data_icd_10;'

In [50]:
index_tester(None, Q, D)

('Seq Scan on "leczenie_w_kierunku_kod_ICD_10"  (cost=0.00..1118.00 rows=1888 width=28) (actual time=0.044..7.996 rows=1889 loops=1)',)
("  Filter: ((data_zakonczenia_leczenia_i10 >= '2021-02-01'::date) AND (data_zakonczenia_leczenia_i10 <= '2021-02-28'::date))",)
('  Rows Removed by Filter: 48111',)
('Planning Time: 4.303 ms',)
('Execution Time: 8.127 ms',)


In [37]:
index_tester('CREATE INDEX data_icd_10 on ai_med."leczenie_w_kierunku_kod_ICD_10"(data_zakonczenia_leczenia_i10)', Q, D)

('Bitmap Heap Scan on "leczenie_w_kierunku_kod_ICD_10"  (cost=27.64..423.96 rows=1888 width=28) (actual time=0.221..0.861 rows=1889 loops=1)',)
("  Recheck Cond: ((data_zakonczenia_leczenia_i10 >= '2021-02-01'::date) AND (data_zakonczenia_leczenia_i10 <= '2021-02-28'::date))",)
('  Heap Blocks: exact=364',)
('  ->  Bitmap Index Scan on data_icd_10  (cost=0.00..27.17 rows=1888 width=0) (actual time=0.171..0.172 rows=1889 loops=1)',)
("        Index Cond: ((data_zakonczenia_leczenia_i10 >= '2021-02-01'::date) AND (data_zakonczenia_leczenia_i10 <= '2021-02-28'::date))",)
('Planning Time: 3.231 ms',)
('Execution Time: 1.000 ms',)


In [38]:
index_tester('CREATE INDEX data_icd_10 on ai_med."leczenie_w_kierunku_kod_ICD_10" using brin (data_zakonczenia_leczenia_i10)', Q, D)

('Seq Scan on "leczenie_w_kierunku_kod_ICD_10"  (cost=0.00..1118.00 rows=1888 width=28) (actual time=0.026..7.380 rows=1889 loops=1)',)
("  Filter: ((data_zakonczenia_leczenia_i10 >= '2021-02-01'::date) AND (data_zakonczenia_leczenia_i10 <= '2021-02-28'::date))",)
('  Rows Removed by Filter: 48111',)
('Planning Time: 4.383 ms',)
('Execution Time: 7.504 ms',)


In [39]:
#Q = 'EXPLAIN ANALYSE SELECT * FROM ai_med.objawy WHERE bol_glowy = TRUE;'
Q = 'EXPLAIN ANALYSE SELECT * FROM ai_med.objawy WHERE bol_glowy is TRUE;'
D = 'DROP INDEX IF EXISTS ai_med.data_bol;'

In [40]:
index_list = [None, 'HASH', 'gist', 'spgist', 'gin', 'BRIN']

In [41]:
for index in index_list:
    print(index)
    index_tester(None, Q, D)

None
('Seq Scan on objawy  (cost=0.00..771.00 rows=25175 width=10) (actual time=0.028..7.822 rows=25045 loops=1)',)
('  Filter: (bol_glowy IS TRUE)',)
('  Rows Removed by Filter: 24955',)
('Planning Time: 3.078 ms',)
('Execution Time: 9.048 ms',)
HASH
('Seq Scan on objawy  (cost=0.00..771.00 rows=25175 width=10) (actual time=0.039..7.862 rows=25045 loops=1)',)
('  Filter: (bol_glowy IS TRUE)',)
('  Rows Removed by Filter: 24955',)
('Planning Time: 2.811 ms',)
('Execution Time: 9.028 ms',)
gist
('Seq Scan on objawy  (cost=0.00..771.00 rows=25175 width=10) (actual time=0.034..7.018 rows=25045 loops=1)',)
('  Filter: (bol_glowy IS TRUE)',)
('  Rows Removed by Filter: 24955',)
('Planning Time: 2.447 ms',)
('Execution Time: 8.120 ms',)
spgist
('Seq Scan on objawy  (cost=0.00..771.00 rows=25175 width=10) (actual time=0.029..7.052 rows=25045 loops=1)',)
('  Filter: (bol_glowy IS TRUE)',)
('  Rows Removed by Filter: 24955',)
('Planning Time: 2.521 ms',)
('Execution Time: 8.179 ms',)
gin
('Seq 

In [42]:
index_tester(None, Q, D)

('Seq Scan on objawy  (cost=0.00..771.00 rows=25175 width=10) (actual time=0.022..7.262 rows=25045 loops=1)',)
('  Filter: (bol_glowy IS TRUE)',)
('  Rows Removed by Filter: 24955',)
('Planning Time: 2.569 ms',)
('Execution Time: 8.481 ms',)


In [43]:
index_tester('CREATE INDEX data_bol on ai_med.objawy(bol_glowy);', Q, D)

('Seq Scan on objawy  (cost=0.00..771.00 rows=25175 width=10) (actual time=0.016..5.819 rows=25045 loops=1)',)
('  Filter: (bol_glowy IS TRUE)',)
('  Rows Removed by Filter: 24955',)
('Planning Time: 4.691 ms',)
('Execution Time: 7.075 ms',)


In [44]:
index_tester('CREATE INDEX data_bol on ai_med.objawy using hash (bol_glowy);', Q, D)

('Seq Scan on objawy  (cost=0.00..771.00 rows=25175 width=10) (actual time=0.015..5.809 rows=25045 loops=1)',)
('  Filter: (bol_glowy IS TRUE)',)
('  Rows Removed by Filter: 24955',)
('Planning Time: 2.901 ms',)
('Execution Time: 6.886 ms',)


In [61]:
Q = 'EXPLAIN ANALYSE SELECT * FROM ai_med."Nazwa pliku z notatkami lekarskimi"' +  " WHERE lekarz_rodzinny LIKE ('%e920%');"
index_name_query = 'plik_test'

D = 'DROP INDEX IF EXISTS ai_med.%s;' %(index_name_query)
index_query = 'CREATE INDEX '
index_query += index_name_query
index_query += ' on ai_med."Nazwa pliku z notatkami lekarskimi" using %s (lekarz_rodzinny);' 

In [62]:
index_tester(None, Q, D)

('Seq Scan on "Nazwa pliku z notatkami lekarskimi"  (cost=0.00..1445.00 rows=5 width=103) (actual time=0.233..11.022 rows=33 loops=1)',)
("  Filter: ((lekarz_rodzinny)::text ~~ '%e920%'::text)",)
('  Rows Removed by Filter: 49967',)
('Planning Time: 2.681 ms',)
('Execution Time: 11.299 ms',)


In [63]:
index_tester(index_query %('btree'), Q, D)

('Seq Scan on "Nazwa pliku z notatkami lekarskimi"  (cost=0.00..1445.00 rows=5 width=103) (actual time=0.165..9.086 rows=33 loops=1)',)
("  Filter: ((lekarz_rodzinny)::text ~~ '%e920%'::text)",)
('  Rows Removed by Filter: 49967',)
('Planning Time: 3.393 ms',)
('Execution Time: 9.156 ms',)


In [64]:
index_tester(index_query %('hash'), Q, D)

('Seq Scan on "Nazwa pliku z notatkami lekarskimi"  (cost=0.00..1445.00 rows=5 width=103) (actual time=0.222..9.270 rows=33 loops=1)',)
("  Filter: ((lekarz_rodzinny)::text ~~ '%e920%'::text)",)
('  Rows Removed by Filter: 49967',)
('Planning Time: 4.186 ms',)
('Execution Time: 9.299 ms',)
